In [8]:
# Pymatgen -> Module for extract data via API Materials Project
from pymatgen.ext.matproj import MPRester

# ETL -> Work with data manipulating/filtering it
import pandas as pd

# Webscraping -> Work with automatic browser
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service

# Work with passwords input
from getpass import getpass

# Work with time
from datetime import time
import time

# Work with math
import math
import numpy as np

# Work with system
import sys


## Definições principais

# Definindo a lista de combinação
metais_de_transicao = ['Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Y','Zr','Nb','Mo','Tc','Ru', 'Rh','Pd','Ag','Cd', 'La','Hf','Ta','W','Re','Os', 'Ir', 'Pt','Au', 'Hg']

# Definindo as combinações permitidas
elementos_permitidos = ['Al', 'B', 'Si']

# Chave da API - Para acessar o Pymatgen
API_KEY = input('Insira o seu token: ')
print('Observação você pode inserir o token diretamente no código no lugar do input para não precisar inseri-lo toda vez,')


## Definindo funções
# Função de mensagem
def mensagem(mensagem):
    '''Envia uma mensagem e verifica a confirmação do usuário para continuar a execução ou parar o programa.'''
    msg = input(mensagem)
    while msg.lower() != 'y':
        msg = input(mensagem)

# Função cabeçalho 
def cabecalho(titulo):
    '''Printa o cabeçalho indicado.'''
    print()
    print('=' * 127)
    print('{:^127}'.format(titulo))
    print('=' * 127)
    print()


# Cabeçalho dos metais de transição
cabecalho('METAIS DE TRANSIÇÃO UTILIZADOS PARA A COMBINAÇÃO')
print(f'''Lista dos metais de transição: [{', '.join(metais_de_transicao)}]

Número total de elementos: {len(metais_de_transicao)}''')

# Cabeçalho do elemento analisado
cabecalho('DEFININDO O ELEMENTO A SER COMBINADO')

# Definindo o elemento químico a ser combinado com os metais de transição
elemento_analisado = input(f'''Olá, seja bem-vindo ao extrator de dados!

Para prosseguir selecione um dos símbolos químicos abaixo para extrair suas combinações com os {len(metais_de_transicao)} metais de transição listados acima.

Os símbolos químicos disponíveis são:
[{', '.join(elementos_permitidos)}]

Insira o símbolo químico desejado: ''')
while elemento_analisado not in elementos_permitidos:
    elemento_analisado = input(f'''Símbolo químico inválido! Os símbolos químicos disponíveis são: [{', '.join(elementos_permitidos)}]
''')
    
# Definindo o nome do arquivo e o diretorio
nome_arquivo_bd = input('Digite o nome do arquivo: ') + '.xlsx'
diretorio = ''

if diretorio == '':
    path = nome_arquivo_bd
else:
    path = diretorio + nome_arquivo_bd

# Cabeçalho da extração via API
cabecalho('EXTRAÇÃO DOS DADOS VIA API')

print('Observação: A partir de agora algumas perguntas para prosseguir serão realizadas, as opções estarão entre colchetes -> [y/n] que siginificam "yes or no" para prosseguir ou não prosseguir.\n')

mensagem(f'Podemos iniciar a extração dos dados dos compostos {elemento_analisado}M? [y/n]')

print('\nExtraindo os dados e gerando o DataFrame...')

start_time = time.time()

df_dados_api = pd.DataFrame() # Creating the Final Database

with MPRester(API_KEY) as mpr:
    for metal in metais_de_transicao:
        formula = f'{elemento_analisado}-{metal}'
        query = mpr.query(formula,['material_id', 'pretty_formula','total_magnetization', 'formation_energy_per_atom' , 'spacegroup','elasticity','density',  'crystal_system', 'volume'])
        df_query = pd.DataFrame(query)
        df_dados_api = df_dados_api.append(df_query)

print(f'''\nDataFrame gerado com sucesso!

Tempo de execução --- {(time.time() - start_time):.2f} seconds ---''')

#----------------------#

display(df_dados_api)

cabecalho('FORMATAÇÃO DO DATAFRAME')

# Renomenado a coluna do volume da célula unitária
df_dados_api.rename(columns = {'volume':'cell_volume'}, inplace = True)

# Resetando os índices do Dataframe
df_dados_api.reset_index(drop=True, inplace=True)

# Criando a Workbook no Excel
writer = pd.ExcelWriter(path, engine='xlsxwriter')

# Criando a Worksheet dentro da Workbook
df_dados_api.to_excel(writer, sheet_name = r'Dados iniciais')

# Gerando as novas colunas
df_dados_api[['spacegroup_number', 'point_group','G_Reuss', 'G_VRH', 'G_Voigt', 'K_Reuss', 'K_VRH', 'K_Voigt', 'elastic_anisotropy', 'universal_anisotropy', 'poisson_ratio', 'Method']] = np.nan


## Formatando os dicionários de Spacegroup e Elasticity em colunas
for n in range(len(df_dados_api)):
    # Separando o dicionário do Spacegroup 
    df_dados_api.loc[df_dados_api.index[n], 'spacegroup_number'] = df_dados_api['spacegroup'][n]['number']
    df_dados_api.loc[df_dados_api.index[n], 'point_group'] = df_dados_api['spacegroup'][n]['point_group']
    
    # Itera para as linhas que possuirem elasticidade
    if df_dados_api.loc[n, 'elasticity'] != None:
        for nova_coluna in ['G_Reuss', 'G_VRH', 'G_Voigt', 'K_Reuss', 'K_VRH', 'K_Voigt', 'elastic_anisotropy', 'universal_anisotropy', 'poisson_ratio']:
            df_dados_api.loc[df_dados_api.index[n], nova_coluna] = df_dados_api['elasticity'][n][nova_coluna]
        
        # Analisa se o id possui avisos e os adiciona em uma nova coluna
        try:
            df_dados_api.loc[df_dados_api.index[n], 'warnings'] = df_dados_api['elasticity'][n]['warnings']
        except ValueError:
            pass
      
        # Extraindo Cij do tensor de elasticidade 
        for i in range(6):
            for j in range(6):
                df_dados_api.loc[df_dados_api.index[n], f'C{i+1}{j+1}'] = df_dados_api['elasticity'][n]['elastic_tensor_original'][i][j]
    
# Retirando as colunas Spacegroup e Elasticity
df_dados_api.drop(columns = ['spacegroup', 'elasticity'], inplace=True)

# Indicando o método
df_dados_api.loc[df_dados_api['K_VRH'].notnull(), 'Method'] = 'DFT'

# Exportando a planilha com as novas colunas
df_dados_api.to_excel(writer, sheet_name='Dados com elasticidade ')

# Fazendo uma cópia
df_id = df_dados_api.copy()
display(df_id)
#----------------------#

# Cabeçalho da extração via Web Scraping
cabecalho('EXTRAÇÃO DOS DADOS VIA WEB SCRAPING')

# Permite o usuário realizar o Web Scraping ou pular, como preferir
webscraping_pergunta = input('''Atenção! Na API estão faltando algumas informações importantes dos compostos:
1. Os dados de K_VRH e G_VRH dos compostos sem matriz de elasticidade, mas que são previstos no site pelos algoritmos de Machine Learning. 
2. A informação 'Decomposes To', que indica para quais compostos o respectivo composto se decompõe. 
Ambas as informações podem ser obtidas por meio de um Web Scraping no site do Materials Project.

Atenção: Se você pular o Web Scraping sua base de dados ficará com essas informações faltantes.

Podemos adicionar à base de dados extraída via API essas novas informações via Web Scraping? [y/n] ''')

while webscraping_pergunta.lower() not in ['y', 'n']:
    webscraping_pergunta = input('''Insira um valor válido! "y" para rodar o Web Scraping ou "n" para pulá-lo.
Atenção: Se você pular o Web Scraping sua base de dados ficará com essas informações faltantes.''')

## Coletando os dados de estabilidade, K_VRH e G_VRH previstos pelo programa de Machine Learning
if webscraping_pergunta.lower() == 'y':

    # Coletando dados com o usuário
    email = input('Insira o seu e-mail usp: ')
    numero_usp = input('Digite seu número usp: ')
    senha_usp = getpass('Digite sua senha usp: ')

    # Outras constantes
    stability_XPATH = '/html/body/div/div/div[3]/div[1]/div[2]/div[2]/table[1]/tbody/tr[6]/td/span'
    
    google_email_xpath = r'//*[@id="identifierId"]'
    google_continuar_xpath = r'//*[@id="identifierId"]'
    google_confirmar_xpath = r'//*[@id="view_container"]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button/span'
    
    num_usp_xpath = r'//*[@id="username"]'
    usp_senha_xpath = r'//*[@id="password"]'
    usp_login_btn_xpath = r'//*[@id="login-main-content"]/div[1]/form/button'
    
    
    url = f'https://materialsproject.org/janrain/loginpage/?next=/janrain/loginpage/'

    ## Iniciando o navegador
    mensagem('Podemos iniciar o navegador? [y/n]')

    start_time = time.time()

    # Inicializando o navegador
    s = Service('chromedriver.exe')
    browser = webdriver.Chrome(service = s)

    browser.get(url) # Navegando para a url indicada
    wait = WebDriverWait(browser, 10)
    action = ActionChains(browser)

    # Fazendo login
    try:
        btn_login_google = wait.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div/div[2]/div[2]/div[1]/a/button')))
        action.move_to_element(btn_login_google).click().perform()

    except TimeoutException:
        print(f'Erro browser demorou muito para carregar ou o elemento não foi encontrado.')
        browser.close()

    wait.until(EC.presence_of_element_located((By.XPATH, google_email_xpath))).send_keys(email)
    wait.until(EC.presence_of_element_located((By.XPATH, google_continuar_xpath))).send_keys(Keys.ENTER)

    wait.until(EC.presence_of_element_located((By.XPATH, num_usp_xpath))).send_keys(numero_usp)
    wait.until(EC.presence_of_element_located((By.XPATH, usp_senha_xpath))).send_keys(senha_usp)
    wait.until(EC.presence_of_element_located((By.XPATH, usp_login_btn_xpath))).click()
    
    wait.until(EC.presence_of_element_located((By.XPATH, google_confirmar_xpath))).click()
    
    home_page_ok = input('Assim que deixar na página principal logada digite ok: ')
    while home_page_ok != 'ok':
        home_page_ok = input('Assim que deixar na página principal logada digite ok: ')
    print()
    
    wait.until(EC.element_to_be_clickable((By.XPATH, r'//*[@id="searchfield"]/form/div[2]/div/button')))

    ## Extraindo estabilidade e predições de K_VRH e G_VRH
    # Extraindo estabilidade 
    for n in range(len(df_id)):
        erro_estabilidade = 0
        material_id = df_id['material_id'][n]

        # Definindo a url
        url = f'https://materialsproject.org/materials/{material_id}/'
        browser.get(url) # Navegando para a url indicada

        try:
            WebDriverWait(browser, 1).until(EC.presence_of_element_located((By.XPATH, stability_XPATH)))
            stability = browser.find_element(By.XPATH, stability_XPATH).text # Valor do elemento web
            df_id.loc[df_id.index[n], 'Decomposes_To'] = stability
            print(f'''Composto: {n}
Id: {material_id}
Decomposes To: {stability}''')

        except TimeoutException:
            print(f'''Composto: {n}
Id: {material_id}
Decomposes To: Não encontrado''')
            erro_estabilidade = 1
            pass

    # Extraindo K_VRH e G_VRH
        K_VRH_material = df_id['K_VRH'][n]

        if erro_estabilidade == 1:
            print(f'K_VRH: Não encontrado, G_VRH: Não encontrado \n')

        elif math.isnan(K_VRH_material):
            try:

                wait.until(EC.element_to_be_clickable((By.XPATH, r'//*[@id="kg-prediction"]/button'))).click()

                K_VRH_predicao = wait.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="kg-prediction"]/div/table/tbody/tr/td[1]/span'))).text
                G_VRH_predicao = wait.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="kg-prediction"]/div/table/tbody/tr/td[2]/span'))).text

                print(f'K_VRH: {K_VRH_predicao}, G_VRH: {G_VRH_predicao} \n')

                K_VRH_predicao = float(K_VRH_predicao[:5])
                G_VRH_predicao = float(G_VRH_predicao[:5])

                df_id.loc[df_id.index[n], 'K_VRH'] = K_VRH_predicao
                df_id.loc[df_id.index[n], 'G_VRH'] = G_VRH_predicao
                df_id.loc[df_id.index[n], 'Method'] = 'ML' # Previstos segundo o algoritmo de Machine Learning

            except TimeoutException:
                print(f'K_VRH: Não encontrado, G_VRH: Não encontrado (TimeoutException)\n')
                pass
        else: # ver de tirar
            df_id.loc[df_id.index[n], 'Method'] = 'DFT' # Previstos segundo o algoritmo de Machine Learning
            print()


    end_time = time.time()

    mensagem('Podemos fechar o navegador? [y/n]')

    # Fechando o navegador
    browser.close()

    print(f'''\nDataFrame editado com sucesso!

Tempo de execução --- {(end_time - start_time):.2f} seconds ---''')

# Gravando a planilha após o Web Scraping
df_id.to_excel(writer, sheet_name='Dados completos')

print('DataFame após a inclusão dos dados coletados pelo Web Scraping:')
display(df_id)

# Cabeçalho da extração via Web Scraping
cabecalho('CALCULANDO OS PARÂMETROS PEDIDOS')

print('Gerando as colunas de Módulo de Young e Coeficente de Poisson...')

# Cálculando o módulo de Young
df_id['E_Young'] = (9 * df_id['K_VRH'] * df_id['G_VRH'])/(3 * df_id['K_VRH'] + df_id['G_VRH']) # Capacidade de resistir a alongamento ou compressão longitudinal

# Cálculando da razão ou coeficiente de Poisson
df_id['Coef_Poisson'] = (3 * df_id['K_VRH'] - df_id['E_Young']) / (6 * df_id['K_VRH']) # Compressão ou alongamento longitudinal

# Criando Worksheet com a coluna de E_Young
df_id.to_excel(writer, sheet_name='Dados calculados')
writer.save()

print()
print(f'\nArquivo {path} salvo com sucesso. Fim da execução!')

cabecalho('VISUALIZAÇÃO DO ARQUIVO FINAL')

display(df_id)

Observação você pode inserir o token diretamente no código no lugar do input para não precisar inseri-lo toda vez,

                                       METAIS DE TRANSIÇÃO UTILIZADOS PARA A COMBINAÇÃO                                        

Lista dos metais de transição: [Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn, Y, Zr, Nb, Mo, Tc, Ru, Rh, Pd, Ag, Cd, La, Hf, Ta, W, Re, Os, Ir, Pt, Au, Hg]

Número total de elementos: 30

                                             DEFININDO O ELEMENTO A SER COMBINADO                                              


                                                  EXTRAÇÃO DOS DADOS VIA API                                                   

Observação: A partir de agora algumas perguntas para prosseguir serão realizadas, as opções estarão entre colchetes -> [y/n] que siginificam "yes or no" para prosseguir ou não prosseguir.


Extraindo os dados e gerando o DataFrame...

DataFrame gerado com sucesso!

Tempo de execução --- 25.07 seconds ---


,material_id,pretty_formula,total_magnetization,formation_energy_per_atom,spacegroup,elasticity,density,crystal_system,volume
0,mp-2252,ScB2,1.814600e-03,-0.838839,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...","{'G_Reuss': 193.0, 'G_VRH': 194.0, 'G_Voigt': ...",3.660544,hexagonal,30.201856
1,mp-8772,ScB12,4.260000e-05,-0.211147,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...","{'G_Reuss': 180.0, 'G_VRH': 188.0, 'G_Voigt': ...",2.850115,cubic,101.776979
2,mp-568384,ScB2,6.327500e-04,0.068805,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...","{'G_Reuss': 135.0, 'G_VRH': 138.0, 'G_Voigt': ...",3.847195,cubic,57.473154
0,mp-1025149,Ti2B,2.864400e-03,-0.394695,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...",None,4.666968,tetragonal,75.818881
1,mp-1245924,TiB12,3.000000e-07,-0.093636,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...",None,2.968304,cubic,99.353065
...,...,...,...,...,...,...,...,...,...
3,mp-1077788,BPt2,3.725000e-05,-0.175723,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...",None,16.985311,hexagonal,78.402015
0,mp-10681,B2Au,7.620000e-04,0.668647,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...",None,10.893524,hexagonal,33.320242
1,mp-1183146,BAu3,2.553000e-04,0.876943,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...",None,15.313583,cubic,65.246921
2,mp-1183149,BAu,1.120000e-05,0.840451,"{'symprec': 0.1, 'source': 'spglib', 'symbol':...",None,10.933356,hexagonal,63.113792



                                                    FORMATAÇÃO DO DATAFRAME                                                    



,material_id,pretty_formula,total_magnetization,formation_energy_per_atom,density,crystal_system,cell_volume,spacegroup_number,point_group,G_Reuss,...,C53,C54,C55,C56,C61,C62,C63,C64,C65,C66
0,mp-2252,ScB2,1.814600e-03,-0.838839,3.660544,hexagonal,30.201856,191.0,6/mmm,193.0,...,-0.000060,0.000000,183.031790,0.000000,0.000020,2.156993e-05,0.000076,0.0,0.000000,221.665290
1,mp-8772,ScB12,4.260000e-05,-0.211147,2.850115,cubic,101.776979,225.0,m-3m,180.0,...,0.000009,0.000000,236.648068,0.000000,0.000002,9.433361e-07,0.000006,0.0,0.000000,236.648067
2,mp-568384,ScB2,6.327500e-04,0.068805,3.847195,cubic,57.473154,227.0,m-3m,135.0,...,0.000000,-0.000019,116.888820,0.000019,0.000000,-3.853589e-05,0.000000,0.0,0.000038,116.888820
3,mp-1025149,Ti2B,2.864400e-03,-0.394695,4.666968,tetragonal,75.818881,140.0,4/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp-1245924,TiB12,3.000000e-07,-0.093636,2.968304,cubic,99.353065,225.0,m-3m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,mp-1077788,BPt2,3.725000e-05,-0.175723,16.985311,hexagonal,78.402015,194.0,6/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,mp-10681,B2Au,7.620000e-04,0.668647,10.893524,hexagonal,33.320242,191.0,6/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,mp-1183146,BAu3,2.553000e-04,0.876943,15.313583,cubic,65.246921,225.0,m-3m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,mp-1183149,BAu,1.120000e-05,0.840451,10.933356,hexagonal,63.113792,186.0,6mm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



                                              EXTRAÇÃO DOS DADOS VIA WEB SCRAPING                                              

DataFame após a inclusão dos dados coletados pelo Web Scraping:


,material_id,pretty_formula,total_magnetization,formation_energy_per_atom,density,crystal_system,cell_volume,spacegroup_number,point_group,G_Reuss,...,C53,C54,C55,C56,C61,C62,C63,C64,C65,C66
0,mp-2252,ScB2,1.814600e-03,-0.838839,3.660544,hexagonal,30.201856,191.0,6/mmm,193.0,...,-0.000060,0.000000,183.031790,0.000000,0.000020,2.156993e-05,0.000076,0.0,0.000000,221.665290
1,mp-8772,ScB12,4.260000e-05,-0.211147,2.850115,cubic,101.776979,225.0,m-3m,180.0,...,0.000009,0.000000,236.648068,0.000000,0.000002,9.433361e-07,0.000006,0.0,0.000000,236.648067
2,mp-568384,ScB2,6.327500e-04,0.068805,3.847195,cubic,57.473154,227.0,m-3m,135.0,...,0.000000,-0.000019,116.888820,0.000019,0.000000,-3.853589e-05,0.000000,0.0,0.000038,116.888820
3,mp-1025149,Ti2B,2.864400e-03,-0.394695,4.666968,tetragonal,75.818881,140.0,4/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp-1245924,TiB12,3.000000e-07,-0.093636,2.968304,cubic,99.353065,225.0,m-3m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,mp-1077788,BPt2,3.725000e-05,-0.175723,16.985311,hexagonal,78.402015,194.0,6/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,mp-10681,B2Au,7.620000e-04,0.668647,10.893524,hexagonal,33.320242,191.0,6/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,mp-1183146,BAu3,2.553000e-04,0.876943,15.313583,cubic,65.246921,225.0,m-3m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,mp-1183149,BAu,1.120000e-05,0.840451,10.933356,hexagonal,63.113792,186.0,6mm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



                                               CALCULANDO OS PARÂMETROS PEDIDOS                                                

Gerando as colunas de Módulo de Young e Coeficente de Poisson...


Arquivo Boreto-teste-14-01.xlsx salvo com sucesso. Fim da execução!

                                                 VISUALIZAÇÃO DO ARQUIVO FINAL                                                 



,material_id,pretty_formula,total_magnetization,formation_energy_per_atom,density,crystal_system,cell_volume,spacegroup_number,point_group,G_Reuss,...,C55,C56,C61,C62,C63,C64,C65,C66,E_Young,Coef_Poisson
0,mp-2252,ScB2,1.814600e-03,-0.838839,3.660544,hexagonal,30.201856,191.0,6/mmm,193.0,...,183.031790,0.000000,0.000020,2.156993e-05,0.000076,0.0,0.000000,221.665290,434.792699,0.120600
1,mp-8772,ScB12,4.260000e-05,-0.211147,2.850115,cubic,101.776979,225.0,m-3m,180.0,...,236.648068,0.000000,0.000002,9.433361e-07,0.000006,0.0,0.000000,236.648067,438.071259,0.165083
2,mp-568384,ScB2,6.327500e-04,0.068805,3.847195,cubic,57.473154,227.0,m-3m,135.0,...,116.888820,0.000019,0.000000,-3.853589e-05,0.000000,0.0,0.000038,116.888820,327.436364,0.186364
3,mp-1025149,Ti2B,2.864400e-03,-0.394695,4.666968,tetragonal,75.818881,140.0,4/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp-1245924,TiB12,3.000000e-07,-0.093636,2.968304,cubic,99.353065,225.0,m-3m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,mp-1077788,BPt2,3.725000e-05,-0.175723,16.985311,hexagonal,78.402015,194.0,6/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,mp-10681,B2Au,7.620000e-04,0.668647,10.893524,hexagonal,33.320242,191.0,6/mmm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,mp-1183146,BAu3,2.553000e-04,0.876943,15.313583,cubic,65.246921,225.0,m-3m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,mp-1183149,BAu,1.120000e-05,0.840451,10.933356,hexagonal,63.113792,186.0,6mm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
